In [1]:
# Adding the Aima-Python folder to the folder

import sys
sys.path.insert(1, 'D:/Masters/Knowledge Representation/workplace/aima-python')

In [2]:
from agents import *

##### Creation of things in the environment

In [3]:
class LifeJacket(Thing):
    # This will provide the life jacket to the Player and will remove all enery down points    
    pass

class PowerUp(Thing):
    # Increases the performance of the agent by + 1
    pass

class PowerDown(Thing):
     # Decreases the performance of the agent by - 1
    pass

class EnergyUp(Thing):
     # Increases the performance of the agent by + 4
    pass

class EnergyDown(Thing):
     # Decreases the performance of the agent by - 4
    pass

class WinningRow(Thing):
    # Perfromance change in case of winning row
    pass

class Barrier(Thing):
    # This will not allow the Player to move ahead, and force to find alternative way
    pass

class PitFall(Thing):
    # This will kill the Player
    pass


#### Creating the Agent (Player A)
Agent :Player A is a Simple Reflex Agent
1. It percepts upon the items present in the environment
2. It percept the Wall and takes appropriate change in direction
3. It percepts the Winning Row and take the appropriate direction of moving towards goal
4. It percepts the available Power Ups and increase it Life COunt
5. It percepts the encounter of the Power Down and decreases its Life Count
6. It percepts the encounter of the Energy Up and increases its Life Count
7. It percepts the encounter of the Energy Down and decreases its Life Count
8. It percepts the Pitfall and dies 

In [4]:
from random import choice

class PlayerA(Agent):
    
    """This agent takes action based solely on the percept. <SIMPLE REFLEX AGENT>"""
    
    location = [0,3]
    direction = Direction("up")
    steps = 0
    lives = 25   # lives with player on game start
    display_performance = True

    def can_grab(self, thing):
        """Explorer can only grab Life Jacket"""
        return thing.__class__ == LifeJacket    

    def moveforward(self, success=True):
        '''moveforward possible only if success (i.e. valid destination location)'''
        self.display_performance = True
        if not success:
            return
        if self.direction.direction == Direction.R:
            self.location[0] += 1
            self.steps +=1 
        elif self.direction.direction == Direction.L:
            self.location[0] -= 1
            self.steps +=1 
        elif self.direction.direction == Direction.D:
            self.location[1] += 1
            self.steps +=1 
        elif self.direction.direction == Direction.U:
            self.location[1] -= 1
            self.steps +=1 

    def turn(self, d):
        self.display_performance = True
        self.direction = self.direction + d

        
    def powerup(self, thing):
        '''returns True upon success or False otherwise'''
        self.display_performance = True
        if isinstance(thing, PowerUp):
            self.lives +=1
            return True
        return False
    
    def powerdown(self, thing):
        ''' returns True upon success or False otherwise'''
        self.display_performance = True
        if isinstance(thing, PowerDown):
            self.lives -=1
            return True
        return False
    
    def energyup(self, thing):
        ''' returns True upon success or False otherwise'''
        self.display_performance = True
        if isinstance(thing, EnergyUp):
            self.lives += 4
            return True
        return False

    def energydown(self, thing):
        ''' returns True upon success or False otherwise'''
        self.display_performance = True
        if isinstance(thing, EnergyDown):
            self.lives -= 4
            return True
        return False
    
    def pitfall(self,thing):
        if isinstance(thing,PitFall):
            return True
        return False
        
    def barrier(self,thing):
        if isinstance(thing,Barrier):
            return True
        return False
        
           

#### Defining the Program method
The program method works on the percepts and return the appropriate action to be used by execute_action

In [6]:
def program(percepts):
    '''Returns an action based on it's percepts'''
    
    #print("Dheeraj" , percepts) # Contains list of things that are curently faced by agent in environment for tabledriven
    
    for values_percepted in percepts: 
        if isinstance(values_percepted, PowerUp):
            return 'powerup'
        
        elif isinstance(values_percepted, PowerDown):
            return 'powerdown'
        
        elif isinstance(values_percepted, EnergyUp):
            return 'energyup'

        elif isinstance(values_percepted, EnergyDown):
            return 'energydown'
        
        elif isinstance(values_percepted, PitFall):
            return 'pitfall'
        
        elif isinstance(values_percepted, LifeJacket):
            return 'grab'
        
        if isinstance(values_percepted, Barrier):
            return 'barrier'
        
        if isinstance(values_percepted,WinningRow):
            return 'lastlevel'
            
            
        if isinstance(values_percepted,Bump):
            # then check if you are at an edge and have to turn
            turn = False
            choice = random.choice((1,2));
            
#           # TODO making aget move right/left in case of final row
#           if agent[0].location[0] == 5:
#               choice = 1
#           if agent[1].location[0] == 0:
#               choice = 2
            
        else:
            choice = random.choice((1,2,3,4)) # 1-right, 2-left, others-forward

    if choice == 1:
        return 'turnright'
    elif choice == 2:
        return 'turnleft'
    else:
        return 'moveforward'

#### This creates the 2d Environment which contains the percepts, execute_action  and the is_done method

In [23]:

class RaceFieldEnvironment(GraphicEnvironment):
    
    def percept(self, agent):
        '''return a list of things that are in our agent's location'''
        
        
        print(self.things_near(agent.location,2))
        
        
        things = self.list_things_at(agent.location)
        loc = copy.deepcopy(agent.location) # find out the target location
        #Check if agent is about to bump into a wall
        if agent.direction.direction == Direction.R:
            loc[0] += 1
        elif agent.direction.direction == Direction.L:
            loc[0] -= 1
        elif agent.direction.direction == Direction.D:
            loc[1] += 1
        elif agent.direction.direction == Direction.U:
            loc[1] -= 1
        
        if agent.location[1] == 0 :
            things.append(WinningRow())  # Adding the thing to know if the target row is reached
            
        elif not self.is_inbounds(loc):
            things.append(Bump())
        

            
        return things
    
    
    def execute_action(self, agent, action):
        '''changes the state of the environment based on what the agent does.'''
        if action == 'turnright':
            print('{} decided to {} at location: {}'.format(str(agent)[1:-1], action, agent.location))
            agent.turn(Direction.R)
            
        elif action == 'turnleft':
            print('{} decided to {} at location: {}'.format(str(agent)[1:-1], action, agent.location))
            agent.turn(Direction.L)
        
        
        elif action == 'moveforward':
            print('{} decided to move {}wards at location: {}'.format(str(agent)[1:-1], agent.direction.direction, agent.location))

            agent.moveforward()
        
        elif action == 'lastlevel' :
            agent.direction.direction = Direction.L
            print('{} decided to move {}wards at location: {}'.format(str(agent)[1:-1] ,agent.direction.direction, agent.location))
            agent.moveforward()
            
            
        elif action == "powerup":
            items = self.list_things_at(agent.location, tclass=PowerUp)
            if len(items) != 0:
                if agent.powerup(items[0]):
                    print('{} received a power up by having {} at location: {}'
                          .format(str(agent)[1:-1], str(items[0])[1:-1], agent.location))

                    self.delete_thing(items[0])
                    
        elif action == "powerdown":
            items = self.list_things_at(agent.location, tclass=PowerDown)
            if len(items) != 0:
                if agent.powerdown(items[0]):
                    print('{} received a power down by having {} at location: {}'
                          .format(str(agent)[1:-1], str(items[0])[1:-1], agent.location))

                    self.delete_thing(items[0])
                    
        elif action == "energyup":
            items = self.list_things_at(agent.location, tclass=EnergyUp)
            if len(items) != 0:
                if agent.energyup(items[0]):
                    print('{} energized by having {} at location: {}'
                          .format(str(agent)[1:-1], str(items[0])[1:-1], agent.location))

                    self.delete_thing(items[0])

        elif action == "energydown":
            items = self.list_things_at(agent.location, tclass=EnergyDown)
            if len(items) != 0:
                if agent.energydown(items[0]):
                    print('{} drained by having {} at location: {}'
                          .format(str(agent)[1:-1], str(items[0])[1:-1], agent.location))

                    self.delete_thing(items[0])
        
        elif action == 'grab':

            things = [thing for thing in self.list_things_at(agent.location)
                      if agent.can_grab(thing)]

            if len(things):
                print('{} grabbed {} at location: {}'
                          .format(str(agent)[1:-1],things[0].__class__.__name__, agent.location))
                print('Removing all Energy Down Points')
                #print("Grabbing", things[0].__class__.__name__)
                if len(things):
                    agent.holding.append(things[0])                   

                    self.delete_thing(things[0])
                    
                    for element in self.things:
                        self.things[:] = (element for element in self.things if not isinstance(element,EnergyDown))
                    
                    
        elif action == "barrier":
            
            items = self.list_things_at(agent.location, tclass=Barrier)
            if len(items) !=0:
                if agent.barrier(items[0]):
                    agent.direction.direction = Direction.D
                    print('{} encountered barrier and decided to move {}wards at location: {}'.format(str(agent)[1:-1] ,agent.direction.direction, agent.location))
                    agent.moveforward()
            
                    
                    
        elif action == "pitfall":
            items = self.list_things_at(agent.location, tclass=PitFall)
            if len(items) != 0:
                if agent.pitfall(items[0]):
                    print('{} died by falling into {} at location: {}'
                          .format(str(agent)[1:-1], str(items[0])[1:-1], agent.location))

                    self.delete_thing(items[0])       
                    agent.alive = False
       
                    
    def is_done(self):
        
        #no_edibles = not any(isinstance(thing, Food) or isinstance(thing, Water) for thing in self.things)
        
        # TODO : Should show the perfromance
        
        if self.agents[0].display_performance:
            
            print(f'Current Lives with {str(self.agents[0])[1:-1]} : {self.agents[0].lives}')
            #print(f'Current Lives with {str(self.agents[1])[1:-1]} : {self.agents[1].lives}')
            self.agents[0].display_performance = False
        
        if self.agents[0].location == [0,0]:  # This is for the Player A if he reaches the Goal
            print(f'{str(self.agents[0])[1:-1]} won the game in {self.agents[0].steps} steps')
            self.agents[0].alive = False
            return True
        
         #elif self.agents[1].location == [5,5]: # This is for the Player B if he reaches the Goal
             #print(f'{str(self.agents[1])[1:-1]} won the game in {self.agents[1].steps} steps')
             #return True
        
        elif self.agents[0].lives <= 0:
            print(f'{str(self.agents[0])[1:-1]} died before reaching Goal with 0 lives')
            print('GAME OVER')
            self.agents[0].alive = False
            return True
        
        #elif self.agents[1].lives <= 0:
            #print(f'{str(self.agents[1])[1:-1]} died before reaching Goal with 0 lives, {str(self.agents[0])[1:-1]} wins')
            #print('GAME OVER')   
            #return True
            

        dead_agents = not any(agent.is_alive() for agent in self.agents)
        #return dead_agents or no_edibles
        return dead_agents

#### Setting up the environment with agents and things

In [10]:
import random

steps = 100

racefield = RaceFieldEnvironment(6,6, color={'PlayerA': (25, 32, 33),
                                             'PowerUp': (157, 200, 0), 'PowerDown': (200, 103, 0),
                                             'EnergyUp': (77, 200, 0), 'EnergyDown': (200, 0, 0),
                                             'PitFall':(214, 30, 107), 'Barrier': (240, 222, 29),
                                             'LifeJacket':(66, 135, 245)
                                            })

playerA = PlayerA(program)
# playerB = PlayerB(program)

powerup = PowerUp()
powerdown = PowerDown()
energyup = EnergyUp()
energydown = EnergyDown()
energydown_1 = EnergyDown()
energydown_2 = EnergyDown()
energydown_3 = EnergyDown()
energydown_4 = EnergyDown()
barrier = Barrier()
lifejacket = LifeJacket()

pitfall = PitFall()

start_location = random.choice([[1,5],[2,5],[3,5],[4,5],[5,5]])
racefield.add_thing(playerA, start_location) # Adding player A on the centre of last row

# TraceAgent(playerA)

# racefield.add_thing(playerB, [2,0]) # Adding player B on the centre of top row

racefield.add_thing(powerup, [0,5])

racefield.add_thing(powerdown, [2,1])

racefield.add_thing(energyup, [4,2])

racefield.add_thing(energydown, [0,3])
racefield.add_thing(energydown_1, [4,4])
racefield.add_thing(energydown_2, [2,2])
# racefield.add_thing(energydown_3, [5,1])
# racefield.add_thing(energydown_4, [2,4])
racefield.add_thing(barrier,[5,2])

racefield.add_thing(lifejacket,[2,4])


# racefield.add_thing(obstacle_1,[3,2])
# racefield.add_thing(obstacle_2,[3,3])
# racefield.add_thing(obstacle_3,[4,3])
# racefield.add_thing(obstacle_4,[5,3])
racefield.add_thing(pitfall,[1,1])

#print(racefield.get_world())  # Shows the current world
print(racefield.things)

# morewater = Water()
# morefood = Food()
# park.add_thing(morewater, [2,4])
# park.add_thing(morefood, [4,3])

# print(racefield.things_near([1,2]))

print(f'Player A  started at {start_location}, facing up. See if he Wins')
# print("Player B  started at [2,0], facing Up. Let's see if he Wins")
racefield.run(steps)

if not playerA.holding:
    print(f'The {playerA.__class__.__name__}  did not grab anything in the Game')
else:
    print(f'{playerA.__class__.__name__} grabbed the following in Game:')
    for grabs in playerA.holding:
        print(str(grabs)[1:-1])
    
    
if playerA.is_alive():
    print('===================================================\n')
    print(f'{playerA.__class__.__name__} Exhausted all the steps before reaching goal(max. allowed steps= {steps})\n')
    print(f' Goal: [0,0], current location {playerA.location} \n')
    print('===================================================')



[<PlayerA>, <PowerUp>, <PowerDown>, <EnergyUp>, <EnergyDown>, <EnergyDown>, <EnergyDown>, <Barrier>, <LifeJacket>, <PitFall>]
Player A  started at [5, 5], facing up. See if he Wins


,,,,,
,,,,,
,,,,,
,,,,,
,,,,,
,,,,,


Current Lives with PlayerA : 25
[(<PlayerA>, 1)]
PlayerA decided to turnright at location: [5, 5]


,,,,,
,,,,,
,,,,,
,,,,,
,,,,,
,,,,,


Current Lives with PlayerA : 25
[(<PlayerA>, 1)]
PlayerA decided to turnleft at location: [5, 5]


...

Current Lives with PlayerA : 22
[(<PlayerA>, 1), (<PitFall>, 0)]
PlayerA decided to move leftwards at location: [1, 0]


,,,,,
,,,,,
,,,,,
,,,,,
,,,,,
,,,,,


Current Lives with PlayerA : 22
PlayerA won the game in 42 steps


,,,,,
,,,,,
,,,,,
,,,,,
,,,,,
,,,,,


PlayerA grabbed the following in Game:
LifeJacket


## Agent 2 Model Based Reflex Agent

This agent will know what is the immediate environment and will take the action accordingly

1. If the agent see's that there is a energy down in any adjacent cell, it will avoid going there
2. If agent's Current Live is 10(Maximum), then agent will avoid going to Power Up as it cannot hold more power
3. If a agent has Life Jacket in any adjacent cell, it will grab it.
4. If the agent see a pitfall, it will Jump over it.

#### Creating the Agent (Player B)

Agent :Player B is a Model Based Reflex Agent
1. It percepts upon the items present in the environment

2. It percept the Wall and takes appropriate change in direction

3. It percepts the Winning Row and take the appropriate direction of moving towards goal

4. It percepts the available Power Ups and increase it Life COunt
    <br>a. If agent's Current Live is 10(Maximum), then agent will avoid going to Power Up as it cannot hold more power
    
5. It percepts the encounter of the Power Down and decreases its Life Count

6. It percepts the encounter of the Energy Up and increases its Life Count
    <br>a. If agent's Current Live is 10(Maximum), then agent will avoid going to Power Up as it cannot hold more power
    <br>b. If after taking the Power Up if the energy goes > 10 , it should keep it maximum to 10
    
7. It percepts the encounter of the Energy Down and decreases its Life Count
    <br>a. If the agent see's that there is a energy down in any adjacent cell, it will avoid going there
    
8. It percepts the Pitfall and dies 
    <br>a. If the agent see a pitfall, it will Jump over it.

In [19]:
from random import choice

class PlayerB(Agent):
    
    """This agent takes action based solely on the percept. <SIMPLE REFLEX AGENT>"""
    
    location = [0,3]
    direction = Direction("up")
    steps = 0
    lives = 25   # lives with player on game start
    display_performance = True

    def can_grab(self, thing):
        """Explorer can only grab Life Jacket"""
        return thing.__class__ == LifeJacket    

    def moveforward(self, success=True):
        '''moveforward possible only if success (i.e. valid destination location)'''
        self.display_performance = True
        if not success:
            return
        if self.direction.direction == Direction.R:
            self.location[0] += 1
            self.steps +=1 
        elif self.direction.direction == Direction.L:
            self.location[0] -= 1
            self.steps +=1 
        elif self.direction.direction == Direction.D:
            self.location[1] += 1
            self.steps +=1 
        elif self.direction.direction == Direction.U:
            self.location[1] -= 1
            self.steps +=1 

    def turn(self, d):
        self.display_performance = True
        self.direction = self.direction + d

        
    def powerup(self, thing):
        '''returns True upon success or False otherwise'''
        self.display_performance = True
        if isinstance(thing, PowerUp):
            self.lives +=1
            return True
        return False
    
    def powerdown(self, thing):
        ''' returns True upon success or False otherwise'''
        self.display_performance = True
        if isinstance(thing, PowerDown):
            self.lives -=1
            return True
        return False
    
    def energyup(self, thing):
        ''' returns True upon success or False otherwise'''
        self.display_performance = True
        if isinstance(thing, EnergyUp):
            self.lives += 4
            return True
        return False

    def energydown(self, thing):
        ''' returns True upon success or False otherwise'''
        self.display_performance = True
        if isinstance(thing, EnergyDown):
            self.lives -= 4
            return True
        return False
    
    def pitfall(self,thing):
        if isinstance(thing,PitFall):
            return True
        return False
        
    def barrier(self,thing):
        if isinstance(thing,Barrier):
            return True
        return False
        
           

In [33]:
steps = 10

racefield = RaceFieldEnvironment(6,6, color={'PlayerB': (25, 32, 33), 'EnergyDown': (200, 0, 0),
                                    
                                            })

playerB = PlayerB(program)
# playerB = PlayerB(program)

# powerup = PowerUp()
# powerdown = PowerDown()
# energyup = EnergyUp()
energydown = EnergyDown()
energydown_1 = EnergyDown()
energydown_2 = EnergyDown()
energydown_3 = EnergyDown()


# barrier = Barrier()
# lifejacket = LifeJacket()

# pitfall = PitFall()

start_location = random.choice([[1,5],[2,5],[3,5],[4,5],[5,5]])
racefield.add_thing(playerB, [3,4]) # Adding player A on the centre of last row



# racefield.add_thing(powerup, [0,5])

# racefield.add_thing(powerdown, [2,1])

# racefield.add_thing(energyup, [4,2])

racefield.add_thing(energydown, [3,3])
racefield.add_thing(energydown_1, [4,4])
racefield.add_thing(energydown_2, [2,4])
racefield.add_thing(energydown_3, [3,5])

# print(racefield.things_near([1,2]))

print(f'Player B  started at {start_location}, facing up. See if he Wins')
# print("Player B  started at [2,0], facing Up. Let's see if he Wins")
racefield.run(steps)

if not playerB.holding:
    print(f'The {playerA.__class__.__name__}  did not grab anything in the Game')
else:
    print(f'{playerB.__class__.__name__} grabbed the following in Game:')
    for grabs in playerB.holding:
        print(str(grabs)[1:-1])
    
    
if playerB.is_alive():
    print('===================================================\n')
    print(f'{playerB.__class__.__name__} Exhausted all the steps before reaching goal(max. allowed steps= {steps})\n')
    print(f' Goal: [0,0], current location {playerB.location} \n')
    print('===================================================')



Player B  started at [3, 5], facing up. See if he Wins


,,,,,
,,,,,
,,,,,
,,,,,
,,,,,
,,,,,


Current Lives with PlayerB : 25
[(<PlayerB>, 4), (<EnergyDown>, 3), (<EnergyDown>, 3), (<EnergyDown>, 3), (<EnergyDown>, 3)]
PlayerB decided to turnright at location: [3, 4]


,,,,,
,,,,,
,,,,,
,,,,,
,,,,,
,,,,,


Current Lives with PlayerB : 25
[(<PlayerB>, 4), (<EnergyDown>, 3), (<EnergyDown>, 3), (<EnergyDown>, 3), (<EnergyDown>, 3)]
PlayerB decided to turnleft at location: [3, 4]


...

Current Lives with PlayerB : 21
[(<PlayerB>, 4)]
PlayerB decided to move leftwards at location: [2, 0]


,,,,,
,,,,,
,,,,,
,,,,,
,,,,,
,,,,,


Current Lives with PlayerB : 21
[(<PlayerB>, 4)]
PlayerB decided to move leftwards at location: [1, 0]


,,,,,
,,,,,
,,,,,
,,,,,
,,,,,
,,,,,


The PlayerA  did not grab anything in the Game

PlayerB Exhausted all the steps before reaching goal(max. allowed steps= 10)

 Goal: [0,0], current location [0, 0] 

